# 🧬 StrandWeaver — SVScribe Training (Colab GPU)

Train **1 XGBoost multiclass classifier** for graph-region structural variant type prediction.
Training uses **SV-dense synthetic data** generated by `generate_training_data.py --graph-training`.

| Class | Label | Description |
|-------|-------|-------------|
| 0 | `none` | No structural variant (background) |
| 1 | `deletion` | Deletion SV |
| 2 | `insertion` | Insertion SV |
| 3 | `inversion` | Inversion SV |
| 4 | `duplication` | Duplication SV |

**Training data layout (5 batches × 40 genomes = 200 genomes):**
```
training_data_sv_dense/
├── sv_dense_1e4/          Moderate SV density (10× baseline), all 4 types
├── sv_dense_2e4/          High SV density (20× baseline), higher repeats
├── sv_dense_3e4/          Very high SV density (30× baseline)
├── sv_dense_ins_focus/    Insertion-enriched (50% insertion, 50% other)
├── sv_dense_inv_dup_focus/ Inversion+duplication-enriched
└── sv_dense_generation_report.json
```

Each genome directory contains:
```
genome_NNNN/
├── haplotype_A.fasta
├── haplotype_B.fasta
├── hifi.fastq
├── sv_truth.json
├── metadata.json
└── graph_training/
    ├── sv_detect_training_gNNNN.csv   ← training rows (this notebook)
    ├── edge_ai_training_gNNNN.csv
    ├── diploid_ai_training_gNNNN.csv
    ├── path_gnn_training_gNNNN.csv
    └── overlap_graph_gNNNN.gfa
```

### 40 Features → 5-class SV type prediction

| # | Feature | Description |
|---|---------|-------------|
| 1–13 | Metadata | genome_id, genome_size, chromosome_id, read_technology, coverage_depth, error_rate, ploidy, gc_content_global, repeat_density_global, heterozygosity_rate, random_seed, generator_version, schema_version |
| 14–18 | Coverage stats | coverage_mean, coverage_std, coverage_median, gc_content, repeat_fraction |
| 19–23 | Graph topology | kmer_diversity, branching_complexity, hic_disruption_score, ul_support, mapping_quality |
| 24–28 | Breakpoint signals | region_length, breakpoint_precision, allele_balance, phase_switch_rate, coverage_cv |
| 29–33 | Distribution stats | coverage_skewness, coverage_kurtosis, coverage_p10, coverage_p90, depth_ratio_flank |
| 34–40 | SV signatures | split_read_count, clip_fraction, bubble_size, path_divergence, ul_spanning, coverage_drop_magnitude, orientation_switch_rate |

### Setup
1. `Runtime → Change runtime type → T4 GPU` (or A100 if available)
2. Upload `training_data_sv_dense/` to Google Drive (or generate on Colab)
3. Run cells top-to-bottom

> **Output:** `sv_detector_model.pkl` — drop into `strandweaver/trained_models/sv_detector/`

## 1. Environment Setup

In [ ]:
# ── Verify GPU ──────────────────────────────────────────────────────
import subprocess, sys, os

result = subprocess.run(
    ['nvidia-smi', '--query-gpu=name,memory.total,driver_version', '--format=csv,noheader'],
    capture_output=True, text=True
)
if result.returncode == 0:
    print(f"✓ GPU detected: {result.stdout.strip()}")
    GPU_AVAILABLE = True
else:
    print("⚠ No GPU detected — XGBoost will use CPU (still fast, just slower HP search)")
    GPU_AVAILABLE = False

# Check RAM
import psutil
ram_gb = psutil.virtual_memory().total / 1e9
print(f"✓ RAM: {ram_gb:.1f} GB {'(High-RAM ✓)' if ram_gb > 20 else '(consider High-RAM runtime)'}")
print(f"✓ Python: {sys.version.split()[0]}")

In [ ]:
# ── Install dependencies ────────────────────────────────────────────
!pip install -q xgboost scikit-learn numpy pandas optuna matplotlib seaborn

# Clone StrandWeaver (dev branch) and install — pull latest on re-runs
!git clone -b dev https://github.com/pgrady1322/strandweaver.git /content/strandweaver 2>/dev/null || \
    (cd /content/strandweaver && git pull origin dev)
!cd /content/strandweaver && pip install -e . -q

print("\n✓ Dependencies installed")

In [ ]:
# ── Mount Google Drive (for data + saving results) ──────────────────
from google.colab import drive
drive.mount('/content/drive')

# ── Directory layout ────────────────────────────────────────────────
GDRIVE_DIR        = '/content/drive/MyDrive/Colab_Training'
SVSCRIBE_OUT      = '/content/svscribe_training'
MODELS_OUT        = '/content/trained_models'
GDRIVE_SVSCRIBE   = os.path.join(GDRIVE_DIR, 'svscribe_models')

for d in [GDRIVE_DIR, SVSCRIBE_OUT, MODELS_OUT]:
    os.makedirs(d, exist_ok=True)

print(f"✓ Google Drive mounted")
print(f"✓ Working dirs created")
print(f"  SVScribe output:  {SVSCRIBE_OUT}")
print(f"  Models output:    {MODELS_OUT}")
print(f"  Drive export:     {GDRIVE_DIR}")

In [ ]:
# ── Verify imports ──────────────────────────────────────────────────
import xgboost as xgb
import numpy as np
import pandas as pd
import time
import json
import pickle
import logging
from pathlib import Path
from collections import Counter

logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s', datefmt='%H:%M:%S')
logger = logging.getLogger('svscribe_training')

SEED = 42

# XGBoost device config
XGB_DEVICE = 'cuda' if GPU_AVAILABLE else 'cpu'
XGB_TREE = 'hist'  # gpu_hist deprecated in XGBoost 2.x — 'hist' auto-uses GPU

print(f"✓ XGBoost {xgb.__version__}")
print(f"✓ NumPy {np.__version__}")
print(f"✓ Pandas {pd.__version__}")
print(f"✓ Device: {XGB_DEVICE}")

## 2. Configuration

Edit the cell below to point to your SV training data.

**Option A (recommended):** Upload the `training_data_sv_dense/` directory to Google Drive.

**Option B:** Generate training data on Colab:
```bash
cd /content/strandweaver
python strandweaver/user_training/generate_training_data.py \
    --genome-size 2000000 -n 40 --sv-density 0.0001 \
    --sv-types deletion insertion inversion duplication \
    --read-types hifi --coverage 30 --graph-training \
    -o /content/sv_training_data --seed 42
```

In [ ]:
# ══════════════════════════════════════════════════════════════════════
#  SVSCRIBE CONFIGURATION — edit these before running
# ══════════════════════════════════════════════════════════════════════

# ── Data source ─────────────────────────────────────────────────────
# Path to the training_data_sv_dense directory on Google Drive or Colab
SV_DATA_DIR = os.path.join(GDRIVE_DIR, 'training_data_sv_dense')

# Batch directories (all should exist under SV_DATA_DIR)
SV_BATCHES = [
    'sv_dense_1e4',           # Moderate SV density (10× baseline)
    'sv_dense_2e4',           # High SV density (20× baseline)
    'sv_dense_3e4',           # Very high SV density (30× baseline)
    'sv_dense_ins_focus',     # Insertion-enriched
    'sv_dense_inv_dup_focus', # Inversion+duplication-enriched
]

# ── SV type label mapping ──────────────────────────────────────────
SV_TYPE_MAP = {
    'none':        0,
    'deletion':    1,
    'insertion':   2,
    'inversion':   3,
    'duplication': 4,
}
SV_TYPE_NAMES = {v: k for k, v in SV_TYPE_MAP.items()}

# ── Features to use for training ───────────────────────────────────
# Metadata columns are useful context features for the model:
# they capture genome characteristics that affect SV signal patterns.
METADATA_FEATURES = [
    'genome_size', 'coverage_depth', 'error_rate',
    'gc_content_global', 'repeat_density_global', 'heterozygosity_rate',
]

# Signal features (from graph region analysis)
SIGNAL_FEATURES = [
    'coverage_mean', 'coverage_std', 'coverage_median',
    'gc_content', 'repeat_fraction',
    'kmer_diversity', 'branching_complexity',
    'hic_disruption_score', 'ul_support', 'mapping_quality',
    'region_length', 'breakpoint_precision', 'allele_balance',
    'phase_switch_rate', 'coverage_cv',
    'coverage_skewness', 'coverage_kurtosis',
    'coverage_p10', 'coverage_p90', 'depth_ratio_flank',
    'split_read_count', 'clip_fraction',
    'bubble_size', 'path_divergence', 'ul_spanning',
    'coverage_drop_magnitude', 'orientation_switch_rate',
]

ALL_FEATURES = METADATA_FEATURES + SIGNAL_FEATURES
SV_LABEL = 'sv_type'

# ── Training hyperparameters ────────────────────────────────────────
N_OPTUNA_TRIALS = 30
N_CV_FOLDS = 5

# ── Validate data directory ────────────────────────────────────────
# NOTE: Uses glob instead of os.listdir() — Drive FUSE caching can
# cause listdir() to return empty/partial results on first access.
import glob as _glob

print(f"SV data directory: {SV_DATA_DIR}")
for batch in SV_BATCHES:
    batch_path = os.path.join(SV_DATA_DIR, batch)
    if os.path.isdir(batch_path):
        # glob is more reliable than os.listdir on Google Drive FUSE
        genome_dirs = sorted(_glob.glob(os.path.join(batch_path, 'genome_*', '')))
        n_genomes = len(genome_dirs)
        if n_genomes == 0:
            # Fallback: count any subdirectory (handles renamed dirs)
            genome_dirs = [
                d for d in _glob.glob(os.path.join(batch_path, '*', ''))
                if os.path.isdir(d)
            ]
            n_genomes = len(genome_dirs)
        print(f"  ✓ {batch}: {n_genomes} genomes")
    else:
        print(f"  ✗ {batch}: NOT FOUND at {batch_path}")

# Check generation report
report_path = os.path.join(SV_DATA_DIR, 'sv_dense_generation_report.json')
if os.path.exists(report_path):
    with open(report_path) as f:
        report = json.load(f)
    for name, batch_info in report.get('batches', {}).items():
        status = batch_info.get('status', 'unknown')
        elapsed = batch_info.get('elapsed_seconds', 0)
        print(f"  {name}: {status} ({elapsed/60:.1f} min)")
else:
    print(f"  ⚠ Generation report not found at {report_path}")

## 3. Load & Merge Training Data
Loads `sv_detect_training_*.csv` from all batches and genomes into a single DataFrame.

In [ ]:
# ── Concatenate all SV detect CSVs ──────────────────────────────────
# Uses glob for CSV discovery — more reliable than os.listdir() on
# Google Drive FUSE mounts (avoids Directory caching issues).
frames = []
batch_counts = {}

for batch in SV_BATCHES:
    batch_dir = os.path.join(SV_DATA_DIR, batch)
    if not os.path.isdir(batch_dir):
        print(f"  ⚠ Skipping {batch} (not found)")
        continue

    # Glob for all sv_detect_training CSVs recursively under this batch
    csv_pattern = os.path.join(batch_dir, '*', 'graph_training', 'sv_detect_training_*.csv')
    csv_files = sorted(_glob.glob(csv_pattern))

    if not csv_files:
        # Fallback: try recursive glob in case directory structure varies
        csv_files = sorted(_glob.glob(os.path.join(batch_dir, '**', 'sv_detect_training_*.csv'), recursive=True))

    batch_frames = []
    for csv_path in csv_files:
        try:
            df = pd.read_csv(csv_path)
            df['batch'] = batch
            batch_frames.append(df)
        except Exception as e:
            print(f"  ⚠ Failed to read {csv_path}: {e}")

    if batch_frames:
        batch_df = pd.concat(batch_frames, ignore_index=True)
        batch_counts[batch] = len(batch_df)
        frames.append(batch_df)
        print(f"  ✓ {batch}: {len(batch_df):,} rows from {len(batch_frames)} files")
    else:
        print(f"  ⚠ {batch}: no CSV files found (pattern: {csv_pattern})")

if not frames:
    raise RuntimeError("No training data found! Check SV_DATA_DIR path and Drive mount.")

df_all = pd.concat(frames, ignore_index=True)
print(f"\n{'═'*50}")
print(f"Total rows: {len(df_all):,}")
print(f"Columns:    {len(df_all.columns)}")
print(f"Batches:    {len(batch_counts)}")
for b, n in batch_counts.items():
    print(f"  {b}: {n:,} ({n/len(df_all)*100:.1f}%)")

## 4. Inspect & Preprocess Data

In [ ]:
# ── Encode SV type labels ───────────────────────────────────────────
df_all['sv_type_encoded'] = df_all[SV_LABEL].map(SV_TYPE_MAP)

# Check for unmapped values
unmapped = df_all[df_all['sv_type_encoded'].isna()][SV_LABEL].unique()
if len(unmapped) > 0:
    print(f"⚠ Unmapped sv_type values: {unmapped}")
    df_all = df_all.dropna(subset=['sv_type_encoded'])
    df_all['sv_type_encoded'] = df_all['sv_type_encoded'].astype(int)

# ── Class distribution ──────────────────────────────────────────────
print("Class distribution:")
for label, count in df_all['sv_type_encoded'].value_counts().sort_index().items():
    pct = count / len(df_all) * 100
    name = SV_TYPE_NAMES.get(label, f'unknown_{label}')
    print(f"  {label} ({name:15s}): {count:>8,} ({pct:5.1f}%)")

# ── Feature summary ────────────────────────────────────────────────
print(f"\nFeature summary ({len(ALL_FEATURES)} features):")
missing = [f for f in ALL_FEATURES if f not in df_all.columns]
if missing:
    print(f"  ⚠ Missing columns: {missing}")
    ALL_FEATURES = [f for f in ALL_FEATURES if f in df_all.columns]
    print(f"  Using {len(ALL_FEATURES)} available features")

print(df_all[ALL_FEATURES].describe().round(3).to_string())

# ── Check for NaN/Inf ──────────────────────────────────────────────
nan_counts = df_all[ALL_FEATURES].isna().sum()
nan_cols = nan_counts[nan_counts > 0]
if len(nan_cols) > 0:
    print(f"\n⚠ Columns with NaN values:")
    for col, cnt in nan_cols.items():
        print(f"  {col}: {cnt:,} NaN ({cnt/len(df_all)*100:.1f}%)")
else:
    print(f"\n✓ No NaN values in features")

In [ ]:
# ── Visualize class balance and key feature distributions ───────────
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('SVScribe Training Data Overview', fontsize=16, fontweight='bold')

# 1. Class distribution
ax = axes[0, 0]
class_counts = df_all['sv_type_encoded'].value_counts().sort_index()
colors = ['#2ecc71', '#e74c3c', '#3498db', '#f39c12', '#9b59b6']
bars = ax.bar([SV_TYPE_NAMES[i] for i in class_counts.index], class_counts.values, color=colors)
ax.set_title('Class Distribution')
ax.set_ylabel('Count')
ax.tick_params(axis='x', rotation=30)

# 2. Coverage distribution by SV type
ax = axes[0, 1]
for sv_code in sorted(df_all['sv_type_encoded'].unique()):
    subset = df_all[df_all['sv_type_encoded'] == sv_code]['coverage_mean']
    ax.hist(subset, bins=50, alpha=0.5, label=SV_TYPE_NAMES[sv_code])
ax.set_title('Coverage Mean by SV Type')
ax.set_xlabel('coverage_mean')
ax.legend(fontsize=8)

# 3. Region length distribution
ax = axes[0, 2]
for sv_code in sorted(df_all['sv_type_encoded'].unique()):
    subset = df_all[df_all['sv_type_encoded'] == sv_code]['region_length']
    ax.hist(subset, bins=50, alpha=0.5, label=SV_TYPE_NAMES[sv_code])
ax.set_title('Region Length by SV Type')
ax.set_xlabel('region_length')
ax.legend(fontsize=8)

# 4. Breakpoint precision
ax = axes[1, 0]
for sv_code in sorted(df_all['sv_type_encoded'].unique()):
    subset = df_all[df_all['sv_type_encoded'] == sv_code]['breakpoint_precision']
    ax.hist(subset, bins=50, alpha=0.5, label=SV_TYPE_NAMES[sv_code])
ax.set_title('Breakpoint Precision by SV Type')
ax.set_xlabel('breakpoint_precision')
ax.legend(fontsize=8)

# 5. Coverage drop magnitude
ax = axes[1, 1]
for sv_code in sorted(df_all['sv_type_encoded'].unique()):
    subset = df_all[df_all['sv_type_encoded'] == sv_code]['coverage_drop_magnitude']
    ax.hist(subset, bins=50, alpha=0.5, label=SV_TYPE_NAMES[sv_code])
ax.set_title('Coverage Drop Magnitude by SV Type')
ax.set_xlabel('coverage_drop_magnitude')
ax.legend(fontsize=8)

# 6. Batch distribution
ax = axes[1, 2]
batch_cts = df_all['batch'].value_counts()
ax.barh(range(len(batch_cts)), batch_cts.values, color='steelblue')
ax.set_yticks(range(len(batch_cts)))
ax.set_yticklabels(batch_cts.index, fontsize=9)
ax.set_title('Rows per Batch')
ax.set_xlabel('Count')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig(os.path.join(SVSCRIBE_OUT, 'sv_data_overview.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✓ Data overview plot saved")

## 5. Train SVScribe Classifier
XGBoost multiclass classifier with **Optuna HP search** + **hybrid resampling** (undersample majority `none` class, oversample minority SV types).

In [ ]:
# ── Hybrid resampling ───────────────────────────────────────────────
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, confusion_matrix
)
from sklearn.utils import resample

optuna.logging.set_verbosity(optuna.logging.WARNING)


def hybrid_resample(X, y, max_majority=50_000, random_state=42):
    """
    Hybrid resampling: undersample majority (none) to max_majority,
    oversample minorities to median class size.
    """
    classes, counts = np.unique(y, return_counts=True)
    class_counts = dict(zip(classes, counts))

    # Cap majority class
    capped = {c: min(n, max_majority) for c, n in class_counts.items()}
    median_size = int(np.median(list(capped.values())))

    X_resampled, y_resampled = [], []
    for cls in classes:
        mask = y == cls
        X_cls, y_cls = X[mask], y[mask]
        target = max(median_size, min(len(X_cls), max_majority))

        if len(X_cls) > target:
            X_rs, y_rs = resample(X_cls, y_cls, n_samples=target,
                                  random_state=random_state, replace=False)
        elif len(X_cls) < target:
            X_rs, y_rs = resample(X_cls, y_cls, n_samples=target,
                                  random_state=random_state, replace=True)
        else:
            X_rs, y_rs = X_cls, y_cls

        X_resampled.append(X_rs)
        y_resampled.append(y_rs)

    return np.vstack(X_resampled), np.concatenate(y_resampled)


def optuna_objective(trial, X, y):
    """Optuna objective for SVScribe multiclass classifier."""
    params = {
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 800),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'gamma': trial.suggest_float('gamma', 0, 5.0),
        'tree_method': XGB_TREE,
        'device': XGB_DEVICE,
        'objective': 'multi:softprob',
        'num_class': 5,
        'eval_metric': 'mlogloss',
        'random_state': SEED,
    }

    model = xgb.XGBClassifier(**params)
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
    scores = cross_val_score(model, X, y, cv=skf, scoring='f1_macro')
    return scores.mean()


# ── Prepare data ────────────────────────────────────────────────────
X = df_all[ALL_FEATURES].fillna(0).values
y = df_all['sv_type_encoded'].values

print(f"Raw data: {X.shape[0]:,} samples × {X.shape[1]} features")
print(f"Class distribution: {dict(Counter(y))}")

# ── Hybrid resampling ──────────────────────────────────────────────
X_balanced, y_balanced = hybrid_resample(X, y, max_majority=50_000)
print(f"\nAfter hybrid resampling: {X_balanced.shape[0]:,} samples")
print(f"Balanced distribution:  {dict(Counter(y_balanced))}")

# ── Optuna HP search ────────────────────────────────────────────────
print(f"\nRunning Optuna ({N_OPTUNA_TRIALS} trials)...")
t0 = time.time()
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: optuna_objective(trial, X_balanced, y_balanced),
               n_trials=N_OPTUNA_TRIALS, show_progress_bar=True)

best_params = study.best_params
best_params['tree_method'] = XGB_TREE
best_params['device'] = XGB_DEVICE
best_params['objective'] = 'multi:softprob'
best_params['num_class'] = 5
best_params['eval_metric'] = 'mlogloss'
best_params['random_state'] = SEED

elapsed_hp = time.time() - t0
print(f"\nBest F1-macro: {study.best_value:.4f} ({elapsed_hp/60:.1f} min)")
print(f"Best params: depth={best_params['max_depth']}, "
      f"lr={best_params['learning_rate']:.4f}, "
      f"n_est={best_params['n_estimators']}")

In [ ]:
# ── K-fold cross-validation with best params ────────────────────────
print(f"Running {N_CV_FOLDS}-fold CV...")
skf = StratifiedKFold(n_splits=N_CV_FOLDS, shuffle=True, random_state=SEED)

cv_acc = []
cv_f1 = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_balanced, y_balanced)):
    X_train, X_val = X_balanced[train_idx], X_balanced[val_idx]
    y_train, y_val = y_balanced[train_idx], y_balanced[val_idx]

    model = xgb.XGBClassifier(**best_params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = model.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='macro')
    cv_acc.append(acc)
    cv_f1.append(f1)
    print(f"  Fold {fold+1}: Accuracy={acc:.4f}, F1-macro={f1:.4f}")

print(f"\nCV Summary: Accuracy={np.mean(cv_acc):.4f}±{np.std(cv_acc):.4f}, "
      f"F1-macro={np.mean(cv_f1):.4f}±{np.std(cv_f1):.4f}")

# ── Train final model on all balanced data ──────────────────────────
print(f"\nTraining final model on all {len(y_balanced):,} samples...")
final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X_balanced, y_balanced)

# Save model
sv_model_dir = os.path.join(MODELS_OUT, 'sv_detector')
os.makedirs(sv_model_dir, exist_ok=True)

pkl_path = os.path.join(sv_model_dir, 'sv_detector_model.pkl')
with open(pkl_path, 'wb') as f:
    pickle.dump(final_model, f)

print(f"\n✓ Saved: {pkl_path}")

# ── Classification report ──────────────────────────────────────────
print(f"\n{'═'*60}")
print("Classification Report (last CV fold):")
print(f"{'═'*60}")
class_names = ['none', 'deletion', 'insertion', 'inversion', 'duplication']
print(classification_report(y_val, y_pred, target_names=class_names, digits=3))

# ── Confusion matrix ───────────────────────────────────────────────
cm = confusion_matrix(y_val, y_pred)
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names, ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
ax.set_title('SVScribe — Confusion Matrix (last CV fold)')
plt.tight_layout()
plt.savefig(os.path.join(SVSCRIBE_OUT, 'sv_confusion_matrix.png'), dpi=150, bbox_inches='tight')
plt.show()

# Save training results
sv_results = {
    'best_params': {k: v for k, v in best_params.items() if k != 'device'},
    'cv_accuracy_mean': float(np.mean(cv_acc)),
    'cv_accuracy_std': float(np.std(cv_acc)),
    'cv_f1_macro_mean': float(np.mean(cv_f1)),
    'cv_f1_macro_std': float(np.std(cv_f1)),
    'n_samples_raw': int(len(y)),
    'n_samples_balanced': int(len(y_balanced)),
    'n_features': len(ALL_FEATURES),
    'feature_names': ALL_FEATURES,
    'sv_type_map': SV_TYPE_MAP,
    'class_distribution_raw': {SV_TYPE_NAMES[k]: int(v) for k, v in Counter(y).items()},
    'class_distribution_balanced': {SV_TYPE_NAMES[k]: int(v) for k, v in Counter(y_balanced).items()},
    'batches_used': SV_BATCHES,
}
with open(os.path.join(sv_model_dir, 'training_results.json'), 'w') as f:
    json.dump(sv_results, f, indent=2, default=str)

print(f"✓ Training results saved")

## 6. Feature Importance

In [ ]:
# ── Feature importance plot ──────────────────────────────────────────
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
fig.suptitle('Feature Importance — SVScribe SV Detector', fontsize=16, fontweight='bold')

importances = final_model.feature_importances_
sorted_idx = np.argsort(importances)
ax.barh(range(len(sorted_idx)), importances[sorted_idx], color='steelblue')
ax.set_yticks(range(len(sorted_idx)))
ax.set_yticklabels([ALL_FEATURES[i] for i in sorted_idx], fontsize=9)
ax.set_xlabel('Importance (gain)')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig(os.path.join(MODELS_OUT, 'svscribe_feature_importance.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✓ Feature importance plot saved")

# Top 10 features
print("\nTop 10 features:")
top10_idx = sorted_idx[-10:][::-1]
for i, idx in enumerate(top10_idx):
    print(f"  {i+1}. {ALL_FEATURES[idx]:30s} {importances[idx]:.4f}")

## 7. Per-Type Precision–Recall Analysis

In [ ]:
# ── Per-SV-type metrics across all CV folds ─────────────────────────
from sklearn.metrics import precision_recall_fscore_support

# Re-run CV to collect per-type metrics
per_type_precision = {c: [] for c in range(5)}
per_type_recall = {c: [] for c in range(5)}
per_type_f1 = {c: [] for c in range(5)}

skf = StratifiedKFold(n_splits=N_CV_FOLDS, shuffle=True, random_state=SEED)
for fold, (train_idx, val_idx) in enumerate(skf.split(X_balanced, y_balanced)):
    X_train, X_val = X_balanced[train_idx], X_balanced[val_idx]
    y_train, y_val = y_balanced[train_idx], y_balanced[val_idx]

    m = xgb.XGBClassifier(**best_params)
    m.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = m.predict(X_val)

    prec, rec, f1, _ = precision_recall_fscore_support(y_val, y_pred, labels=list(range(5)))
    for c in range(5):
        per_type_precision[c].append(prec[c])
        per_type_recall[c].append(rec[c])
        per_type_f1[c].append(f1[c])

# Plot
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.suptitle('Per-Type Metrics (5-fold CV)', fontsize=14, fontweight='bold')

for ax, metric, metric_name in zip(axes, [per_type_precision, per_type_recall, per_type_f1],
                                    ['Precision', 'Recall', 'F1']):
    means = [np.mean(metric[c]) for c in range(5)]
    stds = [np.std(metric[c]) for c in range(5)]
    ax.bar(class_names, means, yerr=stds, capsize=5, color=colors, edgecolor='black', linewidth=0.5)
    ax.set_title(metric_name)
    ax.set_ylim(0, 1.05)
    ax.tick_params(axis='x', rotation=30)

plt.tight_layout(rect=[0, 0, 1, 0.92])
plt.savefig(os.path.join(SVSCRIBE_OUT, 'sv_per_type_metrics.png'), dpi=150, bbox_inches='tight')
plt.show()

# Table
print(f"\n{'SV Type':<15}  {'Precision':>12}  {'Recall':>12}  {'F1':>12}")
print('─' * 55)
for c in range(5):
    print(f"{class_names[c]:<15}  "
          f"{np.mean(per_type_precision[c]):.3f}±{np.std(per_type_precision[c]):.3f}  "
          f"{np.mean(per_type_recall[c]):.3f}±{np.std(per_type_recall[c]):.3f}  "
          f"{np.mean(per_type_f1[c]):.3f}±{np.std(per_type_f1[c]):.3f}")

## 8. Export Models to Google Drive

In [ ]:
# ── Package and export to Google Drive ────────────────────────────────
import tarfile, shutil, glob

GDRIVE_TARBALL = os.path.join(GDRIVE_DIR, 'svscribe_models.tar.gz')

# List all model files
model_files = glob.glob(f'{sv_model_dir}/**/*', recursive=True)
print(f"Model files to export ({len(model_files)}):")
for f in sorted(model_files):
    if os.path.isfile(f):
        size_kb = os.path.getsize(f) / 1024
        print(f"  {os.path.relpath(f, sv_model_dir):40s} {size_kb:>8.1f} KB")

# Create tarball
print(f"\nCreating tarball...")
with tarfile.open(GDRIVE_TARBALL, 'w:gz') as tar:
    tar.add(sv_model_dir, arcname='sv_detector')

tarball_size = os.path.getsize(GDRIVE_TARBALL) / (1024 * 1024)
print(f"\n✅ Exported to Google Drive:")
print(f"   {GDRIVE_TARBALL}")
print(f"   Size: {tarball_size:.1f} MB")

# Also copy to Drive for easy access
if os.path.exists(GDRIVE_SVSCRIBE):
    shutil.rmtree(GDRIVE_SVSCRIBE)
shutil.copytree(sv_model_dir, GDRIVE_SVSCRIBE)
print(f"   Copied → {GDRIVE_SVSCRIBE}")

print(f"\n📥 To download locally:")
print(f"   1. Open Google Drive → My Drive/Colab_Training/")
print(f"   2. Download svscribe_models.tar.gz")
print(f"   3. Extract to strandweaver/trained_models/sv_detector/")

## 9. Summary

In [ ]:
# ── Final summary ────────────────────────────────────────────────────
print("╔══════════════════════════════════════════════════════════════════╗")
print("║         SVScribe Model Training — Summary                       ║")
print("╠══════════════════════════════════════════════════════════════════╣")
print("║                                                                  ║")
print("║  1 XGBoost Multiclass Classifier:                                ║")
print("║  ────────────────────────────────                                 ║")
print(f"║    sv_detector_model       F1={sv_results['cv_f1_macro_mean']:.3f}±"
      f"{sv_results['cv_f1_macro_std']:.3f}  "
      f"Acc={sv_results['cv_accuracy_mean']:.3f}    ║")
print("║                                                                  ║")
print("║  Classes: none | deletion | insertion |                          ║")
print("║           inversion | duplication                                ║")
print("║                                                                  ║")
print(f"║  Training data:                                                  ║")
print(f"║    Raw samples:      {sv_results['n_samples_raw']:>8,}                            ║")
print(f"║    Balanced samples: {sv_results['n_samples_balanced']:>8,}                            ║")
print(f"║    Features:         {sv_results['n_features']:>8}                            ║")
print(f"║    Batches:          {len(SV_BATCHES):>8}                            ║")
print("║                                                                  ║")
print("║  SV density tiers:                                               ║")
print("║    sv_dense_1e4          10× baseline, all 4 SV types            ║")
print("║    sv_dense_2e4          20× baseline, higher repeats            ║")
print("║    sv_dense_3e4          30× baseline                            ║")
print("║    sv_dense_ins_focus    Insertion-enriched                       ║")
print("║    sv_dense_inv_dup_focus Inversion+duplication-enriched          ║")
print("║                                                                  ║")
print("║  Output files on Google Drive:                                   ║")
print(f"║    {GDRIVE_DIR}/")
print(f"║    ├── svscribe_models/     (1 .pkl + results JSON)")
print(f"║    └── svscribe_models.tar.gz")
print("║                                                                  ║")
print("║  Next steps:                                                     ║")
print("║    1. Download sv_detector_model.pkl                             ║")
print("║    2. Place in strandweaver/trained_models/sv_detector/          ║")
print("║    3. Pipeline auto-loads: 'SVDetectorAI loaded'                 ║")
print("║                                                                  ║")
print("╚══════════════════════════════════════════════════════════════════╝")